Follow this instruction to clean data:<br>https://www.kaggle.com/code/hastishahhosseini/usa-real-estate-price-regression

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Read in the data- US Housing Prices
data = pd.read_csv('realtor-data.csv')
data.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [5]:
# Filter the data to California and for sale
data_ca = data[(data['state'] == 'California') & (data['status'] == 'for_sale')].copy()
len(data_ca)

101034

In [6]:
# Convert the 'brokered_by', 'street', and 'zip_code' columns to string
data_ca['brokered_by'] = data_ca['brokered_by'].astype(str)
data_ca['street'] = data_ca['street'].astype(str)
data_ca['zip_code'] = data_ca['zip_code'].astype(str)

In [7]:
# Cleaning data
# Replace NaN values in the 'prev_sold_date' column with "not sold before"
data_ca['prev_sold_date'] = data_ca['prev_sold_date'].fillna("not sold before")

# Remove rows with NaN values in the 'price' column
data_ca_clean = data_ca.dropna(subset=['price'])

# Remove rows with more than 2 NaN values
data_ca_clean = data_ca_clean[data_ca_clean.isna().sum(axis = 1) < 3]

# Fill NaN values in 'house_size' with the median value of 'house_size'
data_ca_clean['house_size'] = data_ca_clean['house_size'].fillna(data_ca_clean['house_size'].median())

# Fill NaN values in 'bath' with the average bath size of similar house size
avg_bath_size = data_ca_clean.groupby('bed')['bath'].mean().round(0).reset_index()
avg_bath_size.columns = ['bed', 'bath_avg']
data_ca_clean = data_ca_clean.merge(avg_bath_size, on = 'bed', how = 'left') #left outer join
data_ca_clean['bath'] = data_ca_clean['bath'].fillna(data_ca_clean['bath_avg'])
# Drop 'bath_avg'
data_ca_clean = data_ca_clean.drop(columns = ['bath_avg'])

# Fill NaN values in 'bed' with the median value of 'bed'
data_ca_clean['bed'] = data_ca_clean['bed'].fillna(data_ca_clean['bed'].median())

In [8]:
data_ca_final = data_ca_clean.dropna(subset = ['price'])

# Filter out outliers based on specified quantiles for 'house_size', 'bed', 'bath', and 'price'
data_ca_final = data_ca_final[
    (data_ca_final['house_size'] > data_ca_final['house_size'].quantile(0.001)) &
    (data_ca_final['house_size'] < data_ca_final['house_size'].quantile(0.99)) &
    (data_ca_final['bed'] > 0) &
    (data_ca_final['bed'] < data_ca_final['bed'].quantile(0.99)) &
    (data_ca_final['bath'] > 0) &
    (data_ca_final['bath'] < data_ca_final['bath'].quantile(0.99)) &
    (data_ca_final['price'] > data_ca_final['price'].quantile(0.001)) &
    (data_ca_final['price'] < data_ca_final['price'].quantile(0.9))
]

In [9]:
#create summary tables based on groupings
# Create a table for the count of rows, sum of price, and average price for each city with non-empty city and state
city_summary = data_ca_final.groupby('city').agg(
    count_rows = ('price', 'size'),
    sum_price = ('price', 'sum'),
    avg_price = ('price', 'mean')
).reset_index()

city_summary = city_summary[city_summary['city'] != ""]  # Filter out empty city

# Create a table for the count of rows, sum of price, and average price for each street
street_summary = data_ca_final.groupby('street').agg(
    count_rows = ('price', 'size'),
    sum_price = ('price', 'sum'),
    avg_price = ('price', 'mean')
).reset_index()

In [10]:
# Merge only the 'avg_price' from city_summary with the main data frame based on 'city' columns
data_ca_final = data_ca_final.merge(city_summary[['city', 'avg_price']], on=['city'], suffixes=('', '_city'))

# Merge only the 'avg_price' from street_summary with the main data frame based on the 'street' column
data_ca_final = data_ca_final.merge(street_summary[['street', 'avg_price']], on='street', suffixes=('', '_street'))

In [11]:
# Calculate the ratio of bedrooms to bathrooms and store it in the 'bed_bath_ratio' column
data_ca_final['bed_bath_ratio'] = data_ca_final['bed'] / data_ca_final['bath']

# Calculate the interaction between house size and number of bathrooms and store it in the 'size_bath_interaction' column
data_ca_final['size_bath_interaction'] = data_ca_final['house_size'] * data_ca_final['bath']

"Given that variables like house size, number of bathrooms, number of bedrooms, the bedroom-to-bathroom ratio, and the size-bath interaction exhibit high correlations with each other, it is imperative to avoid multicollinearity in our model. To achieve this, we will include only the size-bath interaction, which manifests the highest correlation among these variables. Additionally, we will omit all features derived from price (as they are directly derived from it and were solely intended for analytical purposes) and columns with minimal correlation with price, such as acreage."

In [13]:
# Drop specified columns from the real_estate_data_final DataFrame
data_ca_final = data_ca_final.drop(columns=["brokered_by", "status", "bed", "bath", "acre_lot",
                                     "street", "zip_code", "house_size", "prev_sold_date",
                                     "avg_price", "avg_price_street", "bed_bath_ratio", "state"])

In [14]:
# Convert 'city' columns to categorical type
data_ca_final.loc[:, 'city'] = data_ca_final.loc[:, 'city'].astype('category')

# Create dummy variables using pandas get_dummies
# Drop the first dummy variable for each category to avoid multicollinearity
final_data = pd.get_dummies(data_ca_final, columns = ['city'], drop_first=True)

# Ensure dummy variables 'city' are 0s and 1s
final_data.update(final_data.filter(like = 'city_').astype(int))

C:\Users\vnguy\AppData\Local\Temp\ipykernel_4152\2650676668.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  final_data.update(final_data.filter(like = 'city_').astype(int))
C:\Users\vnguy\AppData\Local\Temp\ipykernel_4152\2650676668.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  final_data.update(final_data.filter(like = 'city_').astype(int))
C:\Users\vnguy\AppData\Local\Temp\ipykernel_4152\2650676668.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

In [15]:
final_data

,price,size_bath_interaction,city_Acton,city_Adelanto,city_Adin,city_Agoura,city_Agoura Hills,city_Agua Dulce,city_Aguanga,city_Ahwahnee,...,city_Yorba Linda,city_Yorkville,city_Yosemite West,city_Yountville,city_Yreka,city_Yuba City,city_Yucaipa,city_Yucca Valley,city_Zamora,city_Zenia
0,328000.0,5109.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,199900.0,1014.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,172999.0,2264.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,79900.0,2544.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,69000.0,1134.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68880,875000.0,5874.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68881,49000.0,360.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68882,298000.0,3500.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68883,951140.0,3756.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split

# Define the features and target variable
X = final_data.drop(columns=['price'])
y = final_data['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')